In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import ranksums, ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.power import TTestIndPower
import statsmodels.stats.multicomp as mc

from sklearn.preprocessing import StandardScaler

import os
import json

In [2]:
with open('data/paths.json','r') as f:
    paths = json.load(f)

data_path = paths["data_path"]

In [4]:
df = pd.read_csv('data/plsda/data.csv', index_col=0, header=[0,1])
df.head()


Sample,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Group,H,H,H,H,H,H,H,H,N,N,N,N,N,N,N,N,N
10.8042,-7.130000e-07,1.310000e-07,-2.640000e-07,0.0,-5.740000e-07,-3.410000e-07,-6.220000e-07,-1.280000e-06,-5.330000e-07,-3.290000e-07,-1.730000e-07,-1.140000e-06,-1.830000e-06,-1.230000e-06,5.410000e-07,-7.560000e-07,2.350000e-08
10.8039,-6.600000e-07,6.710000e-07,5.180000e-08,0.0,-4.260000e-08,-2.810000e-07,-4.660000e-07,-1.470000e-06,-7.810000e-07,-1.860000e-07,-1.680000e-08,-9.650000e-07,-1.240000e-06,-1.080000e-06,3.480000e-07,-6.360000e-07,-5.540000e-08
10.8035,-6.280000e-07,6.260000e-07,1.980000e-07,0.0,3.710000e-07,-8.090000e-08,-1.400000e-07,-1.500000e-06,-6.430000e-07,-1.110000e-07,3.380000e-07,-7.090000e-07,-8.500000e-07,-8.010000e-07,1.590000e-07,-4.130000e-07,-1.830000e-07
10.8031,-3.380000e-07,1.190000e-07,3.320000e-08,0.0,2.470000e-07,2.450000e-08,5.630000e-08,-1.210000e-06,-3.040000e-07,-1.130000e-07,5.040000e-07,-4.480000e-07,-6.530000e-07,-3.410000e-07,6.240000e-09,-1.950000e-07,-1.350000e-07
10.8028,-7.320000e-07,4.000000e-08,8.600000e-08,0.0,1.340000e-07,1.360000e-07,8.130000e-09,-6.220000e-07,-2.390000e-08,-7.170000e-08,2.700000e-07,-2.010000e-07,-3.000000e-07,-1.560000e-09,-8.300000e-08,-3.330000e-08,-2.650000e-09


In [5]:
df.columns.get_level_values('Group')

Index(['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N'],
      dtype='object', name='Group')

In [6]:
colormap = {
    'N': '#ff0000',  # Red
    'H': '#0000ff',  # Blue
}

colorlist = [colormap[c] for c in df.columns.get_level_values('Group')]

In [8]:
y = [g == 'N' for g in df.columns.get_level_values('Group')]
y

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [41]:
df = pd.DataFrame({'sampleNum':[1,1,1,1,2,2,2,2,3,3,3,3],'lipid':['a','b','c','d','a','b','c','d','a','b','c','e'],'weight':[11.2,34.2,56.2,12.6,67.5,44.3,78.5,9.4,32.9,33.2,28.4,47.8],'class':[1,1,1,1,0,0,0,0,1,1,1,1]})
df

,sampleNum,lipid,weight,class
0,1,a,11.2,1
1,1,b,34.2,1
2,1,c,56.2,1
3,1,d,12.6,1
4,2,a,67.5,0
5,2,b,44.3,0
6,2,c,78.5,0
7,2,d,9.4,0
8,3,a,32.9,1
9,3,b,33.2,1


In [42]:
colormap = {
    0: '#ff0000',  # Red
    1: '#0000ff',  # Blue
}

colorlist = [colormap[c] for c in df['class']]

In [45]:
st = df.pivot(index='lipid', columns=['sampleNum'], values='weight').reset_index()
st.head()

sampleNum,lipid,1,2,3
0,a,11.2,67.5,32.9
1,b,34.2,44.3,33.2
2,c,56.2,78.5,28.4
3,d,12.6,9.4,NaN
4,e,NaN,NaN,47.8


In [21]:
from sklearn.cross_decomposition import PLSRegression
plsr = PLSRegression(n_components=2, scale=False) # (1)

temp = df[['sampleNum','lipid','weight']]
y = df['class']

plsr.fit(temp, y) 

ValueError: could not convert string to float: 'a'